In [ ]:
!pip install PySCF
!pip install tqdm_joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 41.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
from math import comb
import numpy as np
from pyscf import gto, scf, ao2mo, fci, lo, symm
from pyscf.fci import spin_op
import pyscf
import scipy.linalg as la
import scipy
import scipy.sparse
from pyscf import symm
import os
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib
from scipy import sparse
from pyscf import fci
import multiprocessing
num_workers = max(1, multiprocessing.cpu_count())
print(num_workers)

8


/usr/local/lib/python3.12/dist-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# # ==== Parameters you can adjust ====
# n_h = 6             # Number of H atoms
# bond_length = 2    # H–H distance in Angstrom
# r = bond_length
# # ==== Build the atom string dynamically ====
# atom_lines = []
# for i in range(n_h):
#     x = i * bond_length
#     atom_lines.append(f"H 0.000000 0.000000  {x:.6f}")

# mol = gto.Mole()
# mol.atom = "\n".join(atom_lines)
# mol.basis = 'sto-6g'
# mol.unit = 'Angstrom'
# mol.verbose = 4
# mol.build()

In [ ]:
# r = 1.128
# mol = gto.Mole()
# mol.atom = f"""
# C      0    0    0
# O     0  0   {r:6f}
# """
# mol.basis = 'sto-6g'
# mol.unit = 'Angstrom'
# mol.build()

r = 1.3264
mol = gto.Mole()
mol.atom = f"""
Be      0    0    0
H     0  0   {r:6f}
H     0  0   -{r:6f}
"""
mol.basis = '6-31g'
mol.unit = 'Angstrom'
mol.build()

In [ ]:
def localize_mos(mol, mo_coeff, method='boys'):
    """
    Localize the MOs in `mo_coeff` using the given method.

    Parameters
    ----------
    mol : pyscf.gto.Mole
        Your molecule object.
    mo_coeff : ndarray (nAO, nMO)
        The canonical MO coefficients from SCF.
    method : str
        One of 'boys', 'pm' (Pipek–Mezey), or 'er' (Edmiston–Ruedenberg).

    Returns
    -------
    loc_coeff : ndarray (nAO, nMO)
        The localized MO coefficients.
    """
    m = method.lower()
    if m == 'boys':
        localizer = lo.Boys(mol, mo_coeff)
    elif m in ('pm', 'pipek-mezey'):
        localizer = lo.PM(mol, mo_coeff)
    elif m in ('er', 'edmiston-ruedenberg'):
        localizer = lo.ER(mol, mo_coeff)
    else:
        raise ValueError(f"Unknown localization method: {method!r}")
    return localizer.kernel()

In [ ]:
# 2) Run RHF
mf = scf.RHF(mol)
mf.kernel()
mf.analyze()
ss, mult = mf.spin_square()
print(f"RHF ⟨S²⟩ = {ss:.6f},  multiplicity = {mult}")

# Print basis and CI dimensions
n_ao  = mol.nao_nr()
n_orb = mf.mo_coeff.shape[1]
dim   = comb(n_orb, mol.nelec[0]) * comb(n_orb, mol.nelec[1])
print(f"Number of AOs = {n_ao},  Number of MOs = {n_orb},  CI dim = {dim}")

# 3) Localize MOs (choose your favorite)
method = 'er'   # try 'boys', 'pm', or 'er'
loc_coeff = localize_mos(mol, mf.mo_coeff, method=method)

# 4) Compute orbital spreads ⟨r²⟩ – ⟨r⟩²
dip = mol.intor_symmetric('int1e_r', comp=3)  # shape (3, nAO, nAO)
r2  = mol.intor_symmetric('int1e_r2')         # shape (nAO, nAO)
nmo = loc_coeff.shape[1]
spreads = np.zeros(nmo)

for i in range(nmo):
    c = loc_coeff[:, i]
    # ⟨r_k⟩ for k = x,y,z
    rx, ry, rz = (c @ dip[k] @ c for k in range(3))
    # ⟨r²⟩
    r2_mean = c @ r2 @ c
    # variance
    spreads[i] = r2_mean - (rx*rx + ry*ry + rz*rz)

print("Orbital spreads (⟨r²⟩ − ⟨r⟩²):")
for i, s in enumerate(spreads, start=1):
    print(f"  Orbital {i:2d}: {s:.6f}")

# 5) Build integrals in localized basis
h1_loc = loc_coeff.T @ mf.get_hcore() @ loc_coeff
eri_loc = ao2mo.kernel(mol, loc_coeff)
h2_loc  = ao2mo.restore(8, eri_loc, nmo)

converged SCF energy = -15.7593328355494
**** MO energy ****
MO #1   energy= -4.67469114108643  occ= 2
MO #2   energy= -0.486754697179297 occ= 2
MO #3   energy= -0.447893962537369 occ= 2
MO #4   energy= 0.0895746352202229 occ= 0
MO #5   energy= 0.0895746352202231 occ= 0
MO #6   energy= 0.20486675376911   occ= 0
MO #7   energy= 0.28915910766063   occ= 0
MO #8   energy= 0.438052658629623  occ= 0
MO #9   energy= 0.471390786676046  occ= 0
MO #10  energy= 0.471390786676047  occ= 0
MO #11  energy= 0.587874629054558  occ= 0
MO #12  energy= 1.21599681958563   occ= 0
MO #13  energy= 1.46898273357995   occ= 0
 ** Mulliken atomic charges  **
charge of    0Be =      0.98630
charge of    1H =     -0.49315
charge of    2H =     -0.49315
Dipole moment(X, Y, Z, Debye):  0.00000, -0.00000,  0.00000
RHF ⟨S²⟩ = 0.000000,  multiplicity = 1
Number of AOs = 13,  Number of MOs = 13,  CI dim = 81796
Orbital spreads (⟨r²⟩ − ⟨r⟩²):
  Orbital  1: 0.267098
  Orbital  2: 3.933416
  Orbital  3: 3.595199
  Orbital  

In [ ]:
# 6) FCI in localized MOs
fci_solver = fci.FCI(mol, loc_coeff)
fci_solver.nroots = 10
E_loc, civec_loc = fci_solver.kernel(h1_loc, h2_loc, nmo, mol.nelec)

print("\n=== FCI Results on Localized MOs ===")
for i, Ei in enumerate(E_loc, start=1):
    ss_val, _ = spin_op.spin_square(civec_loc[i-1], nmo, mol.nelec)
    print(f"State {i:2d}: Energy = {Ei:.12f}  ⟨S²⟩ = {ss_val:.6f}")


=== FCI Results on Localized MOs ===
State  1: Energy = -15.800226834150  ⟨S²⟩ = 0.000000
State  2: Energy = -15.572591526189  ⟨S²⟩ = 2.000000
State  3: Energy = -15.567150021911  ⟨S²⟩ = 0.000000
State  4: Energy = -15.538678536636  ⟨S²⟩ = 2.000000
State  5: Energy = -15.538678536633  ⟨S²⟩ = 2.000000
State  6: Energy = -15.471231109444  ⟨S²⟩ = 0.000000
State  7: Energy = -15.471231109399  ⟨S²⟩ = 0.000000
State  8: Energy = -15.430112063863  ⟨S²⟩ = 2.000000
State  9: Energy = -15.394774527749  ⟨S²⟩ = 0.000000
State 10: Energy = -15.343594771338  ⟨S²⟩ = 2.000000


In [ ]:
print('correlation energy',mf.e_tot - E_loc[0] )

correlation energy 0.04089399860042775


In [ ]:
# Extract atomic symbols from the molecule definition
atom_labels = [atom[0] for atom in mol._atom]  # Extract atomic symbols while keeping order
molecule_name = "".join(atom_labels)

# Absorb the 1-electron integrals into the 2-electron integrals
h2e = fci.direct_spin1.absorb_h1e(h1_loc, h2_loc, nmo, mol.nelec, .5)
dets = int(scipy.special.comb(nmo, mol.nelec[0]) * scipy.special.comb(nmo, mol.nelec[1]))

# Define the save folder
if IN_COLAB:
    save_folder = f'/content/drive/My Drive/Quantum_chemistry/es/ham_pieces'
else:
    save_folder = os.path.join(os.getcwd(), "ham_pieces")

os.makedirs(save_folder, exist_ok=True)  # Ensure the directory exists

# Define chunk size (adjust based on available runtime)
chunk_size = 200000  # Change based on available memory
num_chunks = (dets + chunk_size - 1) // chunk_size  # Total number of chunks

# Function to compute a row of the Hamiltonian matrix in CSR format
def compute_H_row(i):
    vector = np.zeros(dets)
    vector[i] = 1.
    H_row = fci.direct_spin1.contract_2e(h2e, vector, nmo, mol.nelec, None)
    H_row[np.isclose(H_row, 0, atol=1e-12)] = 0
    return sparse.csr_matrix(H_row)  # Return as CSR format

# Function to process a chunk of the Hamiltonian matrix
def process_chunk(chunk_idx):
    chunk_file = os.path.join(save_folder, f"{molecule_name}_bond_{r}_Ham_{mol.basis}_chunk_{chunk_idx}.npz")

    if os.path.exists(chunk_file):
        print(f"Chunk {chunk_idx} already exists, skipping computation.")
        return None

    start_idx = chunk_idx * chunk_size
    end_idx = min((chunk_idx + 1) * chunk_size, dets)

    with tqdm_joblib(tqdm(desc=f"Processing chunk {chunk_idx}", total=end_idx - start_idx)):
        chunk_rows = Parallel(n_jobs=num_workers, backend="loky")(
            delayed(compute_H_row)(i) for i in range(start_idx, end_idx)
        )

    # Stack csr matrices efficiently
    chunk_ham = sparse.vstack(chunk_rows).tocsr()

    # Save the chunk
    sparse.save_npz(chunk_file, chunk_ham)
    print(f"Chunk {chunk_idx} saved.")

# Process chunks in parallel
for chunk_idx in range(num_chunks):
    process_chunk(chunk_idx)

# Merge saved chunks into the final Hamiltonian
chunk_files = [os.path.join(save_folder, f"{molecule_name}_bond_{r}_Ham_{mol.basis}_chunk_{i}.npz")
               for i in range(num_chunks) if os.path.exists(
               os.path.join(save_folder, f"{molecule_name}_bond_{r}_Ham_{mol.basis}_chunk_{i}.npz"))]

if len(chunk_files) == num_chunks:
    print("All chunks exist, merging them.")

    # Load and stack all chunks
    ham_chunks = [sparse.load_npz(chunk) for chunk in chunk_files]
    ham = sparse.vstack(ham_chunks).tocsr()

    # Save the final Hamiltonian
    final_file = os.path.join(save_folder, f"{molecule_name}_bond_{r}_Ham_{mol.basis}_local_MO.npz")
    sparse.save_npz(final_file, ham)
    print(f"Final Hamiltonian matrix saved to: {final_file}")

    # Optionally, remove chunk files after merging
    for chunk in chunk_files:
        os.remove(chunk)
    print("Chunk files deleted after merging.")
else:
    print(f"Some chunks are missing. Computed chunks: {len(chunk_files)}/{num_chunks}. Complete missing chunks before merging.")


Processing chunk 0:   0%|          | 0/81796 [00:00<?, ?it/s]

  0%|          | 0/81796 [00:00<?, ?it/s]

Processing chunk 0:   0%|          | 0/81796 [26:46<?, ?it/s]


Chunk 0 saved.
All chunks exist, merging them.
Final Hamiltonian matrix saved to: /content/drive/My Drive/Quantum_chemistry/es/ham_pieces/BeHH_bond_1.3264_Ham_6-31g_local_MO.npz
Chunk files deleted after merging.


In [ ]:
# # read

# string_name = 'HHHHHHHHHHHH_bond_4_Ham.npz'
# if IN_COLAB:
#     file_path = f'/content/drive/My Drive/Quantum_chemistry/es/{string_name}'

# else:
#     file_path = string_name

# h_sparse = scipy.sparse.load_npz(file_path)
# # h_sparse = scipy_to_torch_sparse(h_sparse)
# # h_sparse = h_sparse.to(device)


In [ ]:
print('density of the Ham is', ham.nnz/(ham.shape[0]*ham.shape[0])*100, "%")

density of the Ham is 0.48019623159994546 %


In [ ]:
# # full eigendecomposition method

# # # Diagonalize the Hamiltonian
# eigenvalues, eigenvectors = la.eigh(Ham)

# # Eigenvalues need to add the nuclear repulsion energy back
# true_gs_energy = eigenvalues[0] + mol.energy_nuc()
# true_gs = eigenvectors.T[0]  # Ground state wavefunction

# ###Define the save path
# save_folder = "/content/drive/My Drive/Quantum_chemistry/es/"
# os.makedirs(save_folder, exist_ok=True)  # Ensure the directory exists

# # Save the ground state energy
# energy_file = os.path.join(save_folder, f"{molecule_name}_bond_{r}_true_gs_energy.csv")
# np.savetxt(energy_file, [true_gs_energy], delimiter=',')

# # Save the ground state wavefunction
# wavefunction_file = os.path.join(save_folder, f"{molecule_name}_bond_{r}_true_gs.csv")
# np.savetxt(wavefunction_file, true_gs, delimiter=',')

# print(f"Ground state energy saved to: {energy_file}")
# print(f"Ground state wavefunction saved to: {wavefunction_file}")

#################################################################

## sparse method

number_of_states = 1
result = sparse.linalg.eigsh(ham, k=number_of_states, which='SA')
### for o2
# result = sparse.linalg.eigsh(ham, k=number_of_states, which='SA',ncv=40,maxiter=50000)
true_gs_energy = result[0][0]
true_gs = result[1].T[0]
print('true gs energy', true_gs_energy)
print('anlytical E differnece ', E_loc[0]-true_gs_energy- mol.energy_nuc())
print('1st state weight is' , true_gs[0]*true_gs[0] )

###Define the save path
if IN_COLAB:
    save_folder = f'/content/drive/My Drive/Quantum_chemistry/es/ham_pieces'
else:
    save_folder = os.path.join(os.getcwd(), "ham_pieces")

os.makedirs(save_folder, exist_ok=True)  # Ensure the directory exists

# Save the ground state energy
energy_file = os.path.join(save_folder, f"{molecule_name}_bond_{r}_true_gs_energy_{mol.basis}_local_MO.csv")
np.savetxt(energy_file, [true_gs_energy], delimiter=',')

# Save the ground state wavefunction
wavefunction_file = os.path.join(save_folder, f"{molecule_name}_bond_{r}_true_gs_{mol.basis}_local_MO.csv")
np.savetxt(wavefunction_file, true_gs, delimiter=',')

# print(f"Ground state energy saved to: {energy_file}")
print(f"Ground state wavefunction saved to: {wavefunction_file}")

In [ ]:
print(result[0][0] )

In [ ]:
print('correlation energy',mf.e_tot -( result[0][0] +mol.energy_nuc()) )